# Extract a spectrum on a given position, with a primary beam correction

This notebook extracts and plot a spectra from a cube on a given position. It applies the primary beam correction on the spectra.

## Initialization

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from spectral_cube import SpectralCube
from matplotlib import pyplot as plt
from compass import datadir, extract_spectrum_from_cube

In [ ]:
%matplotlib widget

## Define the source, the frequency setting and the spectral window

In [ ]:
source = "bhr71"
pos = SkyCoord("12h01m34.007s", "-65d08m48.08s", frame="icrs")  # IRS2
setting = 1
spw = 25

## Read the cube and extract a spectrum

In [ ]:
cube = SpectralCube.read(
    datadir + "{}/cubes/{}-set{}-spw{}-lines.fits".format(source, source, setting, spw)
)
restfreq = cube.header["RESTFRQ"] * u.Hz
spectrum = extract_spectrum_from_cube(cube, pos)
spectrum_with_correction = extract_spectrum_from_cube(
    cube, pos, primary_beam_correction=True
)

## Plot the spectrum

In [ ]:
fig, ax = plt.subplots(
    num="{} setting {} spw {}".format(source, setting, spw), clear=True
)
ax.step(
    spectrum.spectral_axis.to(u.GHz, equivalencies=u.doppler_radio(restfreq)).value,
    spectrum.to(u.K).value,
    label="Without correction",
    zorder=2,
)
ax.step(
    spectrum_with_correction.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq)
    ).value,
    spectrum_with_correction.to(u.K).value,
    label="With correction",
    zorder=1,
)
baseline = np.zeros_like(spectrum.spectral_axis.value)
ax.plot(
    spectrum.spectral_axis.to(u.GHz, equivalencies=u.doppler_radio(restfreq)).value,
    baseline,
    color="black",
    linestyle="dotted",
)
ax.legend()
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel("Brightness temperature (K)")